In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъяты лексемы с частотой выше 5.txt
    ├── 1ё_Изъяты лексемы с частотой выше 3.txt
    ├── 2а_ без сокращений.txt
    ├── 2б_Изъяты лексемы с частотой выше 100.txt
    ├── 2в_Изъяты лексемы с ч

In [2]:
ls

 __pycache__/                          loss_comparison.png
 confronto_prestazioni.png             model_results.json
 frecuencia.png                        model_results_GPT.json
 frecuencia_lineas.png                 parte_2.png
 frecuencia_profesional.png            parte_2_Step1.png
 frecuencia_profesional_ajustada.png   partes_discurso.png
 frecuencia_step_1.png                 performance_comparison_other.png
 frecuencia_test_1.png                 pos_analysis.png
 freq_analysis.png                     utils.py
 freq_comparison.png                   wandb/
 funciones.py                         'сокращение по частотности'/
 generador.ipynb                      'сокращение по частотности.ipynb'
 herramientas.ipynb


In [3]:
import json

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

In [ ]:
import funciones
from utils import train_wrapper
import warnings
import os
import torch
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
import multiprocessing as mp
import numpy as np
#import wandb
import nbformat
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gc
import re
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW
from razdel import sentenize
import numpy as np

# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
mp.set_start_method('spawn', force=True)


import json

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)
# import funciones
# from utils import train_wrapper
# import warnings
# import os
# import torch
# import matplotlib.pyplot as plt

# # Suprimir warnings específicos
# warnings.filterwarnings('ignore', category=UserWarning)  # Para sklearn y otros
# warnings.filterwarnings('ignore', category=FutureWarning)  # Para huggingface y transformers
# from concurrent.futures import ProcessPoolExecutor
# from tqdm import tqdm
# from contextlib import redirect_stderr
# from concurrent.futures import ProcessPoolExecutor
# import multiprocessing as mp
# import numpy as np
# mp.set_start_method('spawn', force=True)
# #os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

###############################################################################
# import wandb
# import nbformat
# # Configurar la clave de la API como variable de entorno
# os.environ["WANDB_API_KEY"] = "ca85316ed713b2425615fb3a613d7eb414c9f57f"  # Reemplaza con tu clave
# # Iniciar wandb sin especificar entity (se detecta automáticamente)
# wandb.init(project="model-clasification")
# wandb.init(
#     settings=wandb.Settings(
#         start_method="thread",
#         timeout=30,
#         sync_dir="/tmp/wandb",  # Usa un directorio temporal
#         disable_code=True       # Mejora la estabilidad
#     )
# )
#os.environ["WANDB_NOTEBOOK_NAME"] = "сокращение по частотности.ipynb"
###############################################################################


def create_custom_config(model_name, model_type, dataset):
    """Crea configuración de entrenamiento adaptativa"""
    common_config = {
#         'dataset_name':dataset.get('name', ''),
        'model_name': model_name,
        'model_type': model_type,
        'num_repeats': 6,
        'test_size': 0.2,
        'threshold': 0.5
    }

    # Configuración específica para GPT
    if model_type == 'gpt':
        return funciones.TrainingConfig(
            **common_config,
            max_length=512,
            batch_size=4,
            epochs=2,
            learning_rate=1e-5
        )

    # Configuraciones basadas en frecuencia
    freq_threshold = dataset.get('freq_threshold')
    if freq_threshold in [100, 49, 29, 9, 5, 3]:
        configs = {
            100: (52, 128, 3, 2e-5),
            49: (60, 128, 4, 3e-5),
            29: (51, 128, 4, 3e-5),
            9: (45, 128, 5, 4e-5),
            5: (150, 32, 6, 5e-5),
            3: (150, 32, 6, 5e-5)
        }
        max_len, batch, epochs, lr = configs[freq_threshold]
        return funciones.TrainingConfig(
            **common_config,
            max_length=max_len,
            batch_size=batch,
            epochs=epochs,
            learning_rate=lr
        )

    # Configuración por nombre de dataset
    dataset_name = dataset.get('name', '')
    if any(x in dataset_name for x in ['1', '2', '3', '4']):
        return funciones.TrainingConfig(
            **common_config,
            max_length=60,
            batch_size=128,
            epochs=6,
            learning_rate=5e-5
        )

    # Configuración por defecto
    return funciones.TrainingConfig(
        **common_config,
        max_length=60,
        batch_size=128,
        epochs=4,
        learning_rate=3e-5
    )

models = [
#         {'model':'DeepPavlov/rubert-base-cased',
#          'name':'DeepPavlov-rubert-base',
#          'type': 'bert'},# Modelo original
#         {'model':'bert-base-multilingual-cased',
#          'name':'BERT multilingual',
#          'type': 'bert'},# BERT multilingüe
        {'model':'distilbert-base-multilingual-cased',
         'name':'distilbert-base-multilingual',
         'type': 'bert'},# Versión ligera de BERT
#         {'model':'roberta-base',
#          'name':'roberta-base', 
#         'type': 'bert'}, # RoBERTa 
        {'model': 'gpt2',
         'name': 'gpt2',
         'type': 'gpt'},  #  GPT model
#         {'model': 'facebook/opt-125m',
#          'name': 'facebook',
#          'type': 'gpt'},  #  GPT model
#         {'model': 'sberbank-ai/rugpt3small_based_on_gpt2',
#          'name': 'rugpt3',
#          'type': 'gpt'}
    ]

datasets = [
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100',
            'type': 'freq',
            'freq': 100
        },
#         {
#             'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
#             'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
#             'name': 'Изъяты лексемы с частотой выше 49',
#             'type': 'freq',
#             'freq': 49
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
#             'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
#             'name': 'Изъяты лексемы с частотой выше 29',
#             'type': 'freq',
#             'freq': 29
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
#             'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
#             'name': 'Изъяты лексемы с частотой выше 9',
#             'type': 'freq',
#             'freq': 9
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
#             'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
#             'name': 'Изъяты лексемы с частотой выше 5',
#             'type': 'freq',
#             'freq': 5
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
#             'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
#             'name': 'Изъяты лексемы с частотой выше 3',
#             'type': 'freq',
#             'freq': 3
#         },

#             {
#             'path1': '../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt',
#             'path2': '../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt',
#             'name': 'Без прилагательных второй жанр',
#             'type': 'pos',
#             'freq': None
#             },
#           {
#             'path1': '../dataset/Сокращение по частям речи/1.Первый жанр исходная выборка.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '1.Первый жанр исходная выборка',
#             'type': 'pos',
#             'freq': None

#         },
#         {
#             'path1': '../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '2.Первый жанр без клауз, включающих наречия',
#             'type': 'pos',
#             'freq': None
#         },
#         {
#             'path1': '../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '3.Первый жанр без клауз, включающих глаголы',
#             'type': 'pos',
#             'freq': None
#         },
#         {
#             'path1': '../dataset/Сокращение по частям речи/4.Первый жанр без клауз, включающих глаголы и наречия.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '4.Первый жанр без клауз, включающих глаголы и наречия',
#             'type': 'pos',
#             'freq': None
#         },
    ]

def main():

    results = []
    for model in models:
        for dataset in datasets:
            config = create_custom_config(model['model'], model['type'], dataset)
            
            # Configurar wandb para este experimento
#             wandb_run = wandb.init(
#                 project="model-comparison",
#                 name=f"{dataset['name']} - {model['name']}",
#                 group=f"{dataset['type']}_{dataset.get('freq', 'base')}",
#                 config={
#                     "model": model['model'],
#                     "model_type": model['type'],
#                     "dataset": dataset['name'],
#                     "type": dataset['type'],
#                     "freq": dataset.get('freq', None),
#                     **config.__dict__
#                 },
#                 reinit=True
#             )
            
            # Entrenar y limpiar memoria
            torch.cuda.empty_cache()
            result = funciones.train_and_evaluate_dataset(
                dataset['path1'],
                dataset['path2'],
                config,
                dataset['name'],
                dataset['type']
                
#                 wandb_run=wandb_run
            )
            results.append(result) 
            
#             wandb.log({
#                 "final_avg_accuracy": result['avg_accuracy'],
#                 "final_std_accuracy": result['std_accuracy'],
#                 "final_avg_loss": result['avg_loss']
#             })
            
            # Crear tablas de resumen
#             summary_table = wandb.Table(columns=["Metric", "Value"])
#             summary_table.add_data("Avg Accuracy", result['avg_accuracy'])
#             summary_table.add_data("Std Accuracy", result['std_accuracy'])
#             summary_table.add_data("Avg Loss", result['avg_loss'])
            
#             wandb.log({
#                 f"summary/{dataset['name']}_{clean_model_name}": summary_table,
#                 "epoch": config.epochs  # Para alinear con otras métricas
#             })
            
#             wandb_run.finish()
        
    
    save_results(results, 'model_results_GPT_bert.json')    
#     funciones.plot_model_performance(datasets,
#                                      models,
#                                      results,
#                                      save_path='frecuencia_profesional.png'
#                                     )

    
    
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


train_Изъяты лексемы с частотой выше 100.csv
test_Изъяты лексемы с частотой выше 100.csv
Train labels sample: tensor([1, 1, 1, 0, 0]), Shape: torch.Size([871])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([190])
Epoch 1, Loss: 0.8994767325265067
Epoch 2, Loss: 0.6768577354294913
Epoch 3, Loss: 0.669078162738255
Epoch 4, Loss: 0.663601951939719
Epoch 5, Loss: 0.6583544782229832
Epoch 6, Loss: 0.6454630408968244
all_preds shape: (190,)
all_labels shape: (190,)
all_probs shape: (190, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([871])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([192])
Epoch 1, Loss: 1.0376296298844474
Epoch 2, Loss: 0.6917891076632908
Epoch 3, Loss: 0.6706198028155735
Epoch 4, Loss: 0.6723543235233852
Epoch 5, Loss: 0.6649869169507708
Epoch 6, Loss: 0.6722741808210101
all_preds shape: (192,)
all_labels shape: (192,)
all_probs shape: (192, 2)
Train labels sample: tensor([0, 0, 1, 1, 1]), Shape: torch.Size([871])
Te

In [ ]:
#batch_size=128

In [ ]:
#display_results()

In [ ]:
import os
import IPython

# Reiniciar el kernel
IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart()"))

# Apagar el kernel después de reiniciar
os._exit(0)